# Field-Keypoints Detection Model Local Training

This notebook trains a YOLOv8x model for football field keypoints detection using your local GPU.

In [9]:
# Install required packages
!pip install ultralytics 
!pip install roboflow

## Imports

In [10]:
from roboflow import Roboflow
from ultralytics import YOLO
import torch
from pathlib import Path
from IPython.display import Image

In [11]:
# Check for GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if device == "cuda":
    print(f"GPU Information: {torch.cuda.get_device_name(0)}")
    print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("No GPU detected. Training will be slower on CPU.")

Using device: cuda
GPU Information: NVIDIA A100 80GB PCIe
Available GPU memory: 85.03 GB


## Download Dataset

Download the football players detection dataset from Roboflow.

In [12]:
import roboflow

rf = roboflow.Roboflow(api_key="KBEdRbRsorBypu3bIuO2")

project = rf.workspace("roboflow-jvuqo").project("football-field-detection-f07vi")
version = project.version(12)

print(project)

loading Roboflow workspace...
loading Roboflow project...
{
  "name": "football-field-detection",
  "type": "keypoint-detection",
  "workspace": "roboflow-jvuqo"
}


In [13]:
try:
    # Download dataset from Roboflow (using the same project as in Kaggle)
    print("Downloading dataset from Roboflow...")
    rf = roboflow.Roboflow(api_key="KBEdRbRsorBypu3bIuO2")
    print("Connected to Roboflow")
    project = rf.workspace("roboflow-jvuqo").project("football-field-detection-f07vi")
    print(f"Found project: {project.name}")
    version = project.version(12)
    print(f"Using version: {version.id}")
    dataset = version.download("yolov8", location="/home/asa/cvTemp/foot/field-detection-dataset")
    print("Dataset downloaded successfully")
except Exception as e:
    print(f"Error downloading dataset: {e}")

Connected to Roboflow
loading Roboflow workspace...
loading Roboflow project...
Found project: football-field-detection
Using version: roboflow-jvuqo/football-field-detection-f07vi/12



Extracting Dataset Version Zip to /home/asa/cvTemp/foot/field-detection-dataset in yolov8:: 100%|██████████| 564/564 [00:00<00:00, 3811.57it/s]

Dataset downloaded successfully


In [14]:
# check the dataset variable
print(dataset)
print("Dataset details:")
print(f"Name: {dataset.name}")
print(f"Version: {dataset.version}")
print(f"Location: {dataset.location}")


Dataset details:
Name: football-field-detection
Version: 12
Location: /home/asa/cvTemp/foot/field-detection-dataset


In [15]:
!sed -i 's|\(train: \).*|\1../train/images|' {dataset.location}/data.yaml
!sed -i 's|\(val: \).*|\1../valid/images|' {dataset.location}/data.yaml

## Train the Model

Initialize and train the YOLOv8x model on the football players detection dataset.

In [16]:
data_yaml_path = Path(dataset.location) / "data.yaml"
models_dir = Path("models")
if not models_dir.exists():
    models_dir.mkdir(parents=True, exist_ok=True)
# Initialize a YOLOv8x-pose model
print("Initializing YOLOv8x-pose model...")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")    
model = YOLO('yolov8x-pose.pt')

# Start training
print("Starting model training...")
results = model.train(
    task="pose",           
    data=str(data_yaml_path),
    epochs=100,
    imgsz=640,              
    batch=4,
    mosaic=0.0,             
    plots=True,            
    device=device,
    project=str(models_dir),
    name="field_detector"
)


Initializing YOLOv8x-pose model...
Using device: cuda
Starting model training...
Ultralytics 8.3.97 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100 80GB PCIe, 81093MiB)
engine/trainer: task=pose, mode=train, model=yolov8x-pose.pt, data=/home/asa/cvTemp/foot/field-detection-dataset/data.yaml, epochs=100, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=models, name=field_detector, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, 

train: Scanning /home/asa/cvTemp/foot/field-detection-dataset/train/labels... 222 images, 0 backgrounds, 0 corrupt: 100%|██████████| 222/222 [00:00<00:00, 722.45it/s]


train: New cache created: /home/asa/cvTemp/foot/field-detection-dataset/train/labels.cache


val: Scanning /home/asa/cvTemp/foot/field-detection-dataset/valid/labels... 30 images, 0 backgrounds, 0 corrupt: 100%|██████████| 30/30 [00:00<00:00, 342.97it/s]

val: New cache created: /home/asa/cvTemp/foot/field-detection-dataset/valid/labels.cache


...

     15/100      10.8G     0.5546      9.805     0.5783     0.4445      1.105          2        640: 100%|██████████| 56/56 [00:18<00:00,  3.10it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.84it/s]

                   all         30         30      0.965          1      0.993      0.864     0.0697     0.0667     0.0161    0.00161



      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/56 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 79.19 GiB of which 6.81 MiB is free. Process 2075287 has 38.45 GiB memory in use. Process 2078572 has 9.33 GiB memory in use. Process 2078808 has 9.55 GiB memory in use. Process 2086523 has 9.33 GiB memory in use. Including non-PyTorch memory, this process has 12.46 GiB memory in use. Of the allocated memory 10.82 GiB is allocated by PyTorch, and 81.65 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Validate the Model

Validate the trained model on the validation dataset.

In [ ]:
# Validate the model
print("Validating the model...")
best_model_path = models_dir / "field_detector" / "weights" / "best.pt"
results = model.val(
    task="pose",           # Set task to pose estimation
    data=str(data_yaml_path),
    device=device
)
print(f"Training complete. Best model saved at: {best_model_path}")
# print(f"Validation results: mAP50-95={val_results.box.map:.3f}, mAP50={val_results.box.map50:.3f}")

## Test the Trained Model

You can run a test prediction on an image to verify the model works correctly.

In [ ]:
# Load the trained model
trained_model = YOLO(best_model_path)

# You can test the model on any football image
# If you have a test image, replace 'path/to/test/image.jpg' with the actual path
test_image = r"/home/asa/cvTemp/foot/field-detection-dataset/test/images/08fd33_2_9_png.rf.96fd438c53b674a398918640574dcf10.jpg"
results = trained_model(test_image)
results[0].show()  # Display the detection results

## Update Configuration for Local Inference

Update your local configuration to use this new model.

In [ ]:
# To use this model in your main_test.py,
# update the MODEL_SRC variable in config.py to point to your newly trained model
print(f"To use this model in your application, update the MODEL_SRC variable in config.py to: {best_model_path}")